<img src='../img/EU-Copernicus-EUM_banner_logo.png' align='right' width='50%'></img>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_2_OLCI_file_structure.ipynb">OLCI file structure >></a>

Copernicus Marine Training Service <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1>Learn OLCI: Introductory</h1></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **[A WEkEO account](https://my.wekeo.eu/user-registration)** if you are using WEkEO
- **[A EUMETSAT Earth Observation Portal account](https://eoportal.eumetsat.int/)** if you are using the Data Store or CODA
  

There are no prerequisite notebooks for this modules.
</div>
<hr>

<div class="alert alert-info" role="alert">

## OLCI data access

</div>

### Learning outcomes

At the end of this notebook you will know;
* which <font color="#138D75">**web portals**</font> you can use to access OLCI data
* how to download data from the EUMETSAT Data Store using the <font color="#138D75">**eumetsat data access (eumdac)**</font> client
* how to download data from WEkEO using the <font color="#138D75">**harmonised data access (hda)**</font> client

### Outline

This notebook will download all of the products required for every other lesson in the OLCI introductory material.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Download data via GUIs](#downloading_GUI)
 1. [The products we need](#product_list)
 1. [Downloading via the Data Store API](#Data_Store_API)
 1. [Downloading via the WEkEO API](#WEKEO_API)

<hr>

In [ ]:
import configparser
import shutil
import os
import json
import zipfile
import eumdac #for downloading via the eumetsat/data-store
from hda import Client #for downloading via wekeo
from IPython.display import YouTubeVideo, HTML

# load config
config = configparser.ConfigParser()
config.read(os.path.join(os.path.dirname(os.getcwd()),"config.ini"))

# Create a download directory for out products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='downloading_GUI'></a>1. Downloading data via GUIs
[Back to top](#TOC_TOP)

</div>

### The EUMETSAT Data Store

The [Data Store](https://data.eumetsat.int) is EUMETSAT's primary pull service for delivering ocean data. 

Access to it is possible through a web user interface (WebUI), and through a series of application programming interfaces (APIs). The Data Store supports browsing, searching and downloading data as well as subscription services. It also provides the link to the Data Tailor for customisation. API access is facilitated by the EUMDAC client (see the [Downloading via the Data Store API](#Data_Store_API) section, below).

#### Links:

* [EUMETSAT Data Store](https://data.eumetsat.int)
* [More information on the Data Store](https://eumetsatspace.atlassian.net/wiki/spaces/DSDS/overview)

In [ ]:
YouTubeVideo('WINakzAZvxw', width=config["nbook"]["v_wd"], height=config["nbook"]["v_ht"])

<hr>

### WEkEO

WEkEO intro....

https://www.wekeo.eu/data \
https://www.wekeo.eu/docs

In [ ]:
YouTubeVideo('UOexmq1LdyY', width=config["nbook"]["v_wd"], height=config["nbook"]["v_ht"])

<hr>

### The Copernicus Ocean Data Access (CODA) Portal

#### Links:

* [CODA](https://coda.eumetsat.int)
* [More information on CODA](https://coda.eumetsat.int/manual/CODA-user-manual.pdf)

In [ ]:
YouTubeVideo('V3NAuafvlFM', width=config["nbook"]["v_wd"], height=config["nbook"]["v_ht"])

<div class="alert alert-info" role="alert">

## 2. <a id='product_list'></a>The products we need for this module
[Back to top](#TOC_TOP)
    
</div>

We need a number of OLCI level-1b and level-2 products for the introductory modules. These products are shown below in a python `list` called *product list*. We can retrieve these products from either the Data Store or the WEkEO product catalogue.

In [ ]:
product_list = [
'S3A_OL_1_EFR____20210717T101015_20210717T101315_20210718T145224_0179_074_122_1980_MAR_O_NT_002.SEN3',
'S3A_OL_2_WFR____20210717T101015_20210717T101315_20210718T221347_0179_074_122_1980_MAR_O_NT_003.SEN3',
'S3A_OL_2_WRR____20210717T095732_20210717T104152_20210718T152419_2660_074_122______MAR_O_NT_003.SEN3'
]

<hr>
<div class="alert alert-info" role="alert">

## 3. <a id='Data_Store_API'></a>Downloading from the Data Store via API
[Back to top](#TOC_TOP)

</div>

https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/overview

**PREAMBLE** on `eumdac`

In order to allow us to download data froom the Data Store via API, we need to provide our credentials. To do this, we need to create a file called `.eumdac_credentials` in our home directory. In this file we need to add the following information exactly as follows;

```
{
"consumer_key": "<your_consumer_key>",
"consumer_secret": "<your_consumer_secret>"
}
```

You must replace <your_consumer_key> and <your_consumer_secret> with the information you extract from https://api.eumetsat.int/api-key/. You will need a [EUMETSAT Earth Observation Portal account](https://eoportal.eumetsat.int/) to access this link, and in order to see the information you must click the "Show hidden fields" button at the bottom of the page.

*Note: your key and secret are permanent, so you only need to do this one, but you should take care to never share them*

Once you have done this, you can read in your credentials as follows. These will be used to generate a time-limited token, which will refresh itself when it expires.

In [ ]:
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

Now we have a token, we can see what OLCI specific collections we have in the Data Store.

In [ ]:
datastore = eumdac.DataStore(token)
for collection_id in datastore.collections:
    if "OLCI" in collection_id.title:
        print(f"Collection ID({collection_id}): {collection_id.title}")

So, for;
* **OLCI Level 1B Full Resolution** we want `collection_id`: **EO:EUM:DAT:0409**.
* **OLCI Level 2 Ocean Colour Reduced Resolution** we want `collection_id`: **EO:EUM:DAT:0408**
* **OLCI Level 2 Ocean Colour Full Resolution** we want `collection_id`: **EO:EUM:DAT:0407**

Lets deal with the level-1B data first. Below we provide the level-1B `collection_id` to our `datastore` object to choose the correct collection.

In [ ]:
collection_id = 'EO:EUM:DAT:0409'
selected_collection = datastore.get_collection(collection_id)

We can now find the product directly using the **product name**, which in this case the first item in our product list.

In [ ]:
selected_product = datastore.get_product(product_id=product_list[0], collection_id=collection_id)

Now we can download the product! It will go into our download directory *(../products)*.

In [ ]:
with selected_product.open() as fsrc, \
        open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
    shutil.copyfileobj(fsrc, fdst)
    print(f'Download of product {selected_product} finished.')

The product is downloaded as a zip file, so lets unzip it and remove the zip.

In [ ]:
with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.startswith(str(selected_product)):
            zip_ref.extract(file, download_dir)
    print(f'Unzipping of product {selected_product} finished.')
os.remove(fdst.name)

You can now check the **products** directory to see that we have the level 1B file. Now, lets pull all the parts together and download both level 2 files in a single loop.

In [ ]:
collection_ids = ["EO:EUM:DAT:0407", "EO:EUM:DAT:0408"]

for product_id, collection_id in zip(product_list[1:],collection_ids):
    print(f"Retrieving: {product_id}")    

    selected_collection = datastore.get_collection(collection_id)
    selected_product = datastore.get_product(product_id=product_id, collection_id=collection_id)
    
    with selected_product.open() as fsrc, \
        open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {selected_product} finished.')

    with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.startswith(str(selected_product)):
                zip_ref.extract(file, download_dir)
        print(f'Unzipping of product {selected_product} finished.')

    os.remove(fdst.name)

End of section. Where next?

<a href="../2_OLCI_advanced/2_1_OLCI_advanced_data_access_eumdac.ipynb">Advanced data access with **eumdac**</a>

<hr>
<div class="alert alert-info" role="alert">

## 4. <a id='WEKEO_API'></a>Downloading from WEkEO via the API
[Back to top](#TOC_TOP)

</div>

https://github.com/ecmwf/hda

In [ ]:
# make credentials file

In [ ]:
c = Client()

In [ ]:
query = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___",
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2021-07-17T10:10:15.000Z",
      "end": "2021-07-17T10:13:15.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "platformname",
      "value": "Sentinel-3"
    },
    {
      "name": "producttype",
      "value": "OL_1_EFR___"
    },
    {
      "name": "timeliness",
      "value": "Non Time Critical"
    }
  ]
}

matches = c.search(query)
print(matches)

End of section, where next?

<a href="../2_OLCI_advanced/2_2_OLCI_advanced_data_access_hda.ipynb">Advanced data access with the WEkEO **HDA**</a>

<hr>

<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>